# Evaluation of simulations

Generate figures based on simulated experiments.

---

Load libs

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
if '../' not in sys.path:
    sys.path.append('../')

In [ ]:
import matplotlib.pyplot as plt
import collections
import argparse
import joblib
import os
import glob

from analysis import Evaluation, MultipleEvaluations

---

Define the directories `OUTPUT_DIR` holding the simulation data, and `PLOT_DIR` to save the plots.

In [ ]:
OUTPUT_DIR = '../temp_pickles/'
PLOT_DIR = '../plots/'

Set the directory holding all experiments of interest.

In [ ]:
exp_dir_name = 'reprod_refactoredCode'
# exp_dir_name = 'reprod_workshop_plots'

Generate all plots for each experiments, as well as a plot comparing all-settings.

In [ ]:
# Initialize the summary for multi-setting comparison
multi_summary = collections.defaultdict(dict)

# Analyze each selected experiment
for fpath in glob.glob(os.path.join(OUTPUT_DIR, exp_dir_name, '*.pkl')):
    print(f"=== Analyzing experiment: {fpath:s}")
    # Extract experiment name
    expname = os.path.split(os.path.splitext(fpath)[0])[1]
    # Load data from pickle file
    data = joblib.load(fpath)

    policy_list = [d['name'] for d in data]
    dat = [d['dat'] for d in data]
    n_trials = len(dat[0])

    print("Make plots of the experiments:")
    evaluation = Evaluation(
        data=dat, 
        plot_dir=os.path.join(PLOT_DIR, exp_dir_name, expname), 
        description=policy_list)

#         print('- make present_discounted_loss...')
#         evaluation.present_discounted_loss(plot=True, save=True)

    print('- make simulation_plot(X)...')
    evaluation.simulation_plot(
        process='X', filename='simulation_infection_summary',
        granularity=0.1, save=True)

    print('- make simulation_plot(H)...')
    evaluation.simulation_plot(
        process='H', filename='simulation_treatment_summary',
        granularity=0.1, save=True)

    print('- make simulation_plot(u)...')
    evaluation.simulation_plot(
        process='u', filename='simulation_control_rate_summary',
        granularity=0.1, save=True)

    # Compute Comparison analysis data
    print("Compute comparison analysis data:")
    summary_tup = evaluation.infections_and_interventions_complete(size_tup=(16, 10), save=True)
    multi_summary['infections_and_interventions'][expname] = summary_tup
    multi_summary['Qs'][expname] = evaluation.data[0][0]['info']['Qx'][0]

# Save data for multi-setting comparison for future reuse (small disk space)
joblib.dump(value={'multi_summary': multi_summary, 'policy_list': policy_list, 'n_trials': n_trials}, 
            filename=os.path.join(PLOT_DIR, exp_dir_name, 'multi_comp_dump.pkl'))

# Comparative analysis
multi_eval = MultipleEvaluations(
    multi_summary, policy_list, n_trials, save_dir=os.path.join(PLOT_DIR, exp_dir_name)
)
multi_eval.compare_infections();